In [7]:
import phrase_entity_extraction as ph

In [8]:
st = "160 THE PROMENADE N".lower()
max_address = ph.return_max_address3(ph.seq, st)
if st.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print()
    print("## ************** NOT THE SAME!!!! *********************##\n")
encoding = ph.encode_from_word_list(ph.w(st))
print("Max Address: ", max_address, "\n\nWORD ARRAY: ", ph.w(st), "\n\nENCODING: ", encoding)
print('\n\n{},"{}"'.format(st.upper(), encoding))


## ************** NOT THE SAME!!!! *********************##

Max Address:  N 

WORD ARRAY:  ['160', 'the', 'promenade', 'n'] 

ENCODING:  [['DIGIT', 'ADR_HEAD'], ['ARTI'], ['WORDWAY'], ['LETTER', 'DIR']]


160 THE PROMENADE N,"[['DIGIT', 'ADR_HEAD'], ['ARTI'], ['WORDWAY'], ['LETTER', 'DIR']]"


In [9]:
ph.return_best_fit(ph.seq, st)

[[3, 4, 1, ['SUITE', '_DIR_'], 'n']]

In [10]:
arr_cands = ph.get_markers(ph.seq, st, ['ADDRESS', 'POBOX', 'SUITE'])
arr_cands

[[0, 1, 1, ['SUITE'], '160'], [3, 4, 1, ['SUITE'], 'n']]

In [11]:
import re
stst = 'st'

re.findall(r'^(st|ste)[a-z]+$', stst)

[]

In [12]:
ph.seq.look_ahead(ph.encode_from_word_list(ph.w("st 100"))).get_next_values()

[]